In [1]:
import os

from dotenv import load_dotenv

In [2]:
# set env vars
load_dotenv("./.env")
os.environ["LANGSMITH_PROJECT"] = "structured_output_tagging"

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="claude-sonnet-4-20250514", model_provider="anthropic")

In [4]:
# define pydantic model
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired informaiton from the following passage.
Only extract the properties mentioned in the 'Classification' function.
Passage:
{input}
"""
)


class Classifiaction(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggessiveness: int = Field(description="How aggressive the text is on a scale from 1 to 10")
    language: str = Field(description="The language the text is written in")


structured_llm = llm.with_structured_output(Classifiaction)

In [5]:
# run inference
input = "I'm so friggin happy to have met you! I think we'll be soulmates!"
input_spanish = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"

prompt = tagging_prompt.invoke(input=input)
response = structured_llm.invoke(prompt)

In [6]:
print(response.model_dump())

{'sentiment': 'positive', 'aggessiveness': 2, 'language': 'English'}
